# Import

In [141]:
%config IPCompleter.greedy=True
%run '../utils/import.ipynb'
%run '../utils/export.ipynb'
%run '../utils/pandas.ipynb'
import datacompy

df = importRound1DatasetFromPath('vasy1.csv')
dfTruth = importRound1DatasetFromPath('../ressources/ground_truth.csv')
uniqueUsersTruth = dfTruth['id_user'].unique()
f = pd.DataFrame(index=uniqueUsersTruth, columns=[i + 1 for i in range(13)]).fillna(value='DEL')

## Compare 2 datasets

In [142]:
slicedDf = sliceDFPerMonths(df)
slicedDfTruth = sliceDFPerMonths(dfTruth)

In [143]:
for index, dfMonth in enumerate(slicedDf[0:]):
    month = index + 1
    print('Month n°' + str(month) + ' ------------')
    compare = datacompy.Compare(
        slicedDfTruth[month-1],
        slicedDf[month-1],
        join_columns=['id_item','price','qty'],
        abs_tol=0, #Optional, defaults to 0
        rel_tol=0, #Optional, defaults to 0
        df1_name='dfTruth', #Optional, defaults to 'df1'
        df2_name='df' #Optional, defaults to 'df2'
    )
    concat = pd.concat([slicedDf[month-1], compare.df2_unq_rows]).drop_duplicates(keep=False)
    concatTruth = pd.concat([slicedDfTruth[month-1], compare.df1_unq_rows]).drop_duplicates(keep=False)
    concat2 = pd.concat([concat,concatTruth])
    grouped_df = concat2.groupby(['id_item', 'price','qty'])

    for key, item in grouped_df:
        if grouped_df.get_group(key).shape[0] == 2 :
            truthUser = grouped_df.get_group(key).iat[1,0].astype(int)
            if truthUser in f.index.values :
                f.at[truthUser,month] = grouped_df.get_group(key).iat[0,0].astype(int)
print(f)

Month n°1 ------------
Month n°2 ------------
Month n°3 ------------
Month n°4 ------------
Month n°5 ------------
Month n°6 ------------
Month n°7 ------------
Month n°8 ------------
Month n°9 ------------
Month n°10 ------------
Month n°11 ------------
Month n°12 ------------
Month n°13 ------------
          1      2      3      4      5      6    7      8      9      10  \
17850  53167    DEL    DEL    DEL    DEL    DEL  DEL    DEL    DEL    DEL   
13047    DEL    DEL    DEL    DEL    DEL  70551  DEL    DEL    DEL    DEL   
12583    DEL  39810  58361    DEL    DEL    DEL  DEL    DEL    DEL    DEL   
13748    DEL    DEL    DEL    DEL    DEL    DEL  DEL    DEL    DEL  63819   
15100    DEL    DEL    DEL    DEL    DEL    DEL  DEL    DEL    DEL    DEL   
15291    DEL    DEL    DEL    DEL  89608    DEL  DEL    DEL    DEL    DEL   
14688  87515    DEL    DEL  79445  87978    DEL  DEL  51604  13851    DEL   
17809  92718    DEL    DEL    DEL    DEL    DEL  DEL    DEL  81420    DEL   
1609

In [144]:
f.sort_index(axis = 0).to_csv("./f-test-1.csv", sep=',', index=True, header=True)